## Single Object Tracking

This notebook is an example of how to perform a single object detection and tracking with DeGirum PySDK.
This single object tracking (SOT) also enables users interactively select an object using just a mouse click to track in the video stream. The selected object is highlighted using a bounding box and pose keypoints.

In [ ]:
import cv2
import degirum as dg, degirum_tools
from degirum_tools.video_support import get_video_stream_properties


inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = ''
device_type = "HAILORT/HAILO8"

video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/WalkingPeople2.mp4"
display_name = "Object Selector Example"

person_det_model_name = "yolov8n_relu6_coco_pose--640x640_quant_hailort_hailo8_1"

# Load person/pose detection model
person_det_model = dg.load_model(
    model_name = person_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type,
    overlay_line_width=1
)

In [ ]:
# create a context to store detections and selected track_id
context = dict(detections=None, track_id=None)


def point_in_rect(x, y, rect):
    """Check if point (x, y) is inside rectangle [x1, y1, x2, y2]."""
    x1, y1, x2, y2 = rect
    return x1 <= x <= x2 and y1 <= y <= y2


def is_object_selected(obj, result):
    """Return 1 when object has track_id matching the context, otherwise return 0."""
    # store detections in context for use in mouse callback
    context["detections"] = result.results
    sel_track_id = context.get("track_id")
    track_id = obj.get("track_id")
    return int(
        sel_track_id is not None and track_id is not None and track_id == sel_track_id
    )


def mouse_callback(event: int, x: int, y: int, flags: int, context: dict):
    """Mouse callback to set the context for object selection"""
    if event == cv2.EVENT_LBUTTONDOWN:
        detections = context.get("detections")
        if detections is not None:
            # look for the object that contains clicked point
            for obj in detections:
                # check if the clicked point is inside the bounding box of the object
                track_id = obj.get("track_id")
                if track_id is not None and point_in_rect(x, y, obj["bbox"]):
                    # if so, remember the track_id in context
                    context["track_id"] = track_id
                    break
            else:
                context["track_id"] = None

In [ ]:

# create object tracker analyzer to track objects
tracker = degirum_tools.ObjectTracker(
    track_thresh=0.35, #Detection confidence threshold for initiating a new track.
    match_thresh=0.9999, #Intersection-over-union (IoU) threshold for matching detections to existing tracks.
    anchor_point=degirum_tools.AnchorPoint.CENTER, #Anchor point on the bounding box used for trail visualization.
    show_overlay=False,
)

# create object selector analyzer to select clicked person
selector = degirum_tools.ObjectSelector(
    top_k=0,
    selection_strategy=degirum_tools.ObjectSelectionStrategies.CUSTOM_METRIC,
    # use custom metric to select the object of interest: object with highest metric value is selected
    custom_metric=is_object_selected,
    metric_threshold=0.5,  #Metric value threshold: if top_k is zero, objects with metric value higher than this threshold are selected. Default 0.
    use_tracking=False,
    show_overlay=False,
)

# attach object tracker and object selector analyzers to person detection model
degirum_tools.attach_analyzers(person_det_model, [tracker, selector])

### Video Inference

In [ ]:
# open display window
with degirum_tools.Display(display_name) as display:
    # perform streaming inference on video source
    for i, result in enumerate(
        degirum_tools.predict_stream(person_det_model, video_source)
    ):
        # show the result on the display
        display.show(result)
        # set mouse callback only once and only when the display is opened
        if i == 0:
            cv2.setMouseCallback(display_name, mouse_callback, context)